In [3]:
import os
import pandas as pd
from functions import *

disable_progress = True  # Set to False to enable progress bar

# Get the paths for input data and output directory
data_path, output_path = get_paths()

# Define parameter ranges for each denoiser
GAUSSIAN_SIGMAS = [2.5, 3, 3.5]
MEDIAN_SIZES = [5, 8, 10]

def main():
    """Main script to evaluate denoisers and save results."""
    all_results = []

    # Test Gaussian filter with different sigma values
    for sigma in tqdm(GAUSSIAN_SIGMAS, disable=disable_progress):
        denoiser_name = "Gaussian"
        denoiser, params = gaussian_filter, {'sigma': sigma}

        # Process images and append results
        denoiser_results = process_images(data_path, num_images=2, denoiser=denoiser, disable_progress=disable_progress, **params)
        for result in denoiser_results:
            result.extend([denoiser_name, f"sigma={sigma}"])
        all_results.extend(denoiser_results)
    print("Gaussian filter results saved.")

    # Test Median filter with different size values
    for size in MEDIAN_SIZES:
        denoiser_name = "Median"
        denoiser, params = median_filter, {'size': size}

        # Process images and append results
        denoiser_results = process_images(data_path, num_images=2, denoiser=denoiser, disable_progress=disable_progress, **params)
        for result in denoiser_results:
            result.extend([denoiser_name, f"size={size}"])
        all_results.extend(denoiser_results)
    print("Median filter results saved.")

    # Convert results to a DataFrame and reorder columns
    all_results_df = pd.DataFrame(all_results, columns=[
        'ImageIndex', 'Channel', 'PSNR', 'SSIM', 'Runtime', 'RAM Usage', 'DenoiserType', 'Parameter'
    ])
    all_results_df = all_results_df[['DenoiserType', 'Parameter', 'ImageIndex', 'Channel', 
                                     'PSNR', 'SSIM', 'Runtime', 'RAM Usage']]

    # Display and save all results
    display_styled_results(all_results_df, output_path, 'all_denoiser_results.csv', title="All Results")

    # Compute averages and save
    avg_results = compute_averages(all_results_df)
    display_styled_results(avg_results, output_path, 'average_denoiser_results.csv', title="Average Results")

if __name__ == "__main__":
    main()

Gaussian filter results saved.
Median filter results saved.

All Results:


,DenoiserType,Parameter,ImageIndex,Channel,PSNR,SSIM,Runtime,RAM Usage
0,Gaussian,sigma=2.5,001,0,30.76,0.8961,0.0031 s,0.59 MB
1,Gaussian,sigma=2.5,001,1,31.25,0.8092,0.0034 s,0.69 MB
2,Gaussian,sigma=2.5,001,2,28.74,0.8009,0.0032 s,0.48 MB
3,Gaussian,sigma=2.5,002,0,36.62,0.9776,0.0034 s,0.95 MB
4,Gaussian,sigma=2.5,002,1,40.51,0.9774,0.0031 s,0.00 MB
5,Gaussian,sigma=2.5,002,2,39.92,0.9648,0.0031 s,0.55 MB
6,Gaussian,sigma=3,001,0,30.70,0.8956,0.0040 s,0.56 MB
7,Gaussian,sigma=3,001,1,31.08,0.8147,0.0036 s,0.52 MB
8,Gaussian,sigma=3,001,2,32.38,0.8373,0.0036 s,0.52 MB
9,Gaussian,sigma=3,002,0,33.79,0.9746,0.0037 s,0.48 MB


Results saved to /Users/Matthieu/Documents/GitHub/ML4Science-CS433/output/all_denoiser_results.csv

Average Results:


,DenoiserType,Parameter,Channel,PSNR,SSIM,Runtime,RAM Usage
0,Gaussian,sigma=2.5,0,33.69,0.9369,0.0033 s,0.77 MB
1,Gaussian,sigma=2.5,1,35.88,0.8933,0.0032 s,0.34 MB
2,Gaussian,sigma=2.5,2,34.33,0.8829,0.0031 s,0.52 MB
3,Gaussian,sigma=3,0,32.25,0.9351,0.0039 s,0.52 MB
4,Gaussian,sigma=3,1,34.24,0.8946,0.0037 s,0.26 MB
5,Gaussian,sigma=3,2,34.05,0.9026,0.0035 s,0.26 MB
6,Gaussian,sigma=3.5,0,33.00,0.9277,0.0040 s,0.05 MB
7,Gaussian,sigma=3.5,1,33.93,0.8904,0.0044 s,0.51 MB
8,Gaussian,sigma=3.5,2,35.95,0.9128,0.0042 s,0.49 MB
9,Median,size=10,0,29.92,0.9181,0.2797 s,0.36 MB


Results saved to /Users/Matthieu/Documents/GitHub/ML4Science-CS433/output/average_denoiser_results.csv
